# 03 - Extract geographical watershed properties

## Extract geographical watershed properties automatically using PAVICS-Hydro's geospatial toolbox

Hydrological models typically need geographical information about watersheds being simulated: latitude and longitude, area, mean altitude, land-use, etc. Raven is no exception. This notebook shows how to obtain this information using remote services that are made available for users in PAVICS-Hydro. These services connect to a digital elevation model (DEM) and a land-use data set to extract relevant information.

The DEM used in the following is the [EarthEnv-DEM90](https://www.earthenv.org/DEM), while the land-use dataset is the [North American Land Change Monitoring System](http://www.cec.org/tools-and-resources/north-american-environmental-atlas/north-american-land-change-monitoring-system). Other data sources could be used, given their availability through the Web Coverage Service (WCS) protocol.

Since these computations happen on a specific Geoserver hosted in PAVICS, we need to establish a connection to that service. While the steps are a bit more complex, the good news is that you only need to change a few items in this notebook to taylor results to your needs.


In [ ]:
# We need to import a few packages required to do the work
import json
import os
from zipfile import ZipFile
from birdy import WPSClient
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import rioxarray as rio
import xarray as xr

# This is the URL of the Geoserver that will perform the computations for us.
url = os.environ.get("WPS_URL", "https://pavics.ouranos.ca/twitcher/ows/proxy/raven/wps")

# Connect to the PAVICS-Hydro Raven WPS server
wps = WPSClient(url)

## Watershed contour / shapefile

We first extract the watershed contour for the point of interest. If you already have the contours in shapefile or GeoJSON format, then you can skip this section and go to the "Display contour" section. 

The process looks into the HydroSHEDS database to finds the watershed enclosing the given location. The `location` parameter identifies the outlet of the watershed, and `aggregate_upstream` determines whether or not we want the service to return the union of all upstream basins. Here we set it to `False` to reduce the size of the basin and speed-up computations.

The output of the `hydrosheds-select` process is a GeoJSON file.

**Run this section to extract a watershed from the HydroSHEDS database:**

In [ ]:
select_resp = wps.hydrobasins_select(
    location="-71.291660, 50.492758", 
    aggregate_upstream=False
)

# Get GeoJSON polygon of the delineated watershed.
# We can either get links to the files stored on the server, or get the data directly.

# Get the links
feature_url, upstream_basins_url = select_resp.get(asobj=False)
print("This is the geoJSON file that can be used as the basin contour in other toolboxes:")
print(feature_url)
print("")

# Get the data directly
feature, upstream_basins = select_resp.get(asobj=True)

We can now plot the outline of the watershed by loading it into `GeoPandas`.

In [ ]:
# If you have a file, you can upload it here (shapefiles must be zipped with the .prj, .shx, .shp, and other such files into a single zip file) and pass the filename.
# Ex: df = gpd.read_file("my_basin_shapefile.zip")

# Here we are using the result of the above computation.
df = gpd.read_file(feature_url)
display(df)
df.plot()

## Generic watershed properties

Now that we have delineated a watershed, lets find the zonal statistics and other properties using the `shape_properties` process. This process requires a `shape` argument defining the watershed contour, the exterior polygon. The polygon can be given either as a link to a geometry file  (e.g. a geojson file such as `feature_url`), or as data embeded in a string. For example, if variable `feature` is a `GeoPandas` geometry, `json.dumps(feature)` can be used to convert it to a string and pass it as the `shape` argument.

Typically, we expect users will simply upload a shapefile and use this code to perform the extraction on the region of interest.

In [ ]:
shape_resp = wps.shape_properties(shape=feature_url)

Once the process has completed, we extract the data from the response, as follows. Note that you do not need to change anything here. The code will work and return the desired results.

In [ ]:
[properties,] = shape_resp.get(asobj=True)
prop = properties[0]
display(prop)

area = prop["area"] / 1000000.0
longitude = prop["centroid"][0]
latitude = prop["centroid"][1]
gravelius = prop["gravelius"]
perimeter = prop["perimeter"]

shape_info = {
    "area": area,
    "longitude": longitude,
    "latitude": latitude,
    "gravelius": gravelius,
    "perimeter": perimeter,
}
display(shape_info)

Note that these properties are a mix of the properties of the original file where the shape is stored, and properties computed by the process (area, centroid, perimeter and gravelius). Note also that the computed area is in m², while the "SUB_AREA" property is in km², and that there are slight differences between the two values due to the precision of HydroSHEDS and the delineation algorithm.

## Land-use information

Now we extract the land-use properties of the watershed using the `nalcms_zonal_stats` process. As mentionned, it uses the [North American Land Change Monitoring System](http://www.cec.org/tools-and-resources/north-american-environmental-atlas/north-american-land-change-monitoring-system) dataset, and retrieve properties over the given region.

With the `nalcms_zonal_stats_raster` process, we also return the grid with variable accessors (`gdal`, `rasterio`, or `rioxarray`) depending on what libraries are available in our runtime environment (The following examples show `rioxarray`-like access).

In [ ]:
stats_resp = wps.nalcms_zonal_stats_raster(
    shape=feature_url, 
    select_all_touching=True, 
    band=1, 
    simple_categories=True
)

Here we will get the raster data and show it as a grid. These values can be exported to the user path to download them and use them offline.

Note that GeoJSON, and rasterio/rioxarray need to be installed for this to work. In the next birdy release, the tiff grid will be automatically converted to a DataArray.

Here we need to manually convert it.

In [ ]:
features, statistics, grid = stats_resp.get(asobj=True)
grid[0].plot()

From there, it's easy to calculate the ratio and percentages of each land-use component. This code should also be left as-is unless you really know what you are doing.

In [ ]:
lu = statistics[0]
total = sum(lu.values())

land_use = {k: (v / total) for (k, v) in lu.items()}
display("Land use ratios", land_use)

precision = 2
land_use_pct = {
    k: "{} %".format(int((v / total) * 100 * 10 ** precision) / 10 ** precision)
    for (k, v) in lu.items()
}
display("Land use percentages", land_use_pct)

### Display the land-use statistics
Here we can display the land-use statistics according to the land cover map, as a function of land cover raster pixels over the catchment. Again, this does not need to be modified at all. It can also be simply deleted if the visualization tools are not required for your use-case.

In [ ]:
unique, counts = np.unique(
    grid[0], 
    return_counts=True
)
display(unique, counts)

# Pixels values at '127' are NaN and can be ignored.
plt.bar(unique[:-1], counts[:-1])
plt.xticks(np.arange(min(unique[:-1]), max(unique[:-1])+1, 1.0))
plt.xlabel('Land-use categories')
plt.ylabel('Number of pixels')
plt.show()

grid[0].where(grid[0] != 127).sel(band=1).plot.imshow(cmap="tab20")
grid[0].name = "Land-use categories"
plt.show()

We can also receive the geotiff object as an `xarray.Dataset` with the `.open_rasterio()` method. This makes it very easy to spatially reproject it with the `cartopy` library. Here we provide a sample projection, but this would need to be adapted to your needs.

In [ ]:
import cartopy.crs as ccrs

# Set a CRS transformation:
crs = ccrs.LambertConformal(
    central_latitude=49, 
    central_longitude=-95, 
    standard_parallels=(49, 77)
)

ax = plt.subplot(projection=crs)
grid[0].name = "Land-use categories"
grid[0].where(grid[0] != 127).sel(band=1).plot.imshow(
    ax=ax, 
    transform=crs, 
    cmap="tab20"
)
plt.show()

## Terrain information from the DEM

Here we collect terrain data, such as elevation, slope and aspect, from the DEM. We will do this using the `terrain_analysis` WPS service, which by default uses DEM data from [EarthEnv-DEM90](https://www.earthenv.org/DEM).

Note here that while the feature outline is defined above in terms of geographic coordinates (latitude, longitude), the DEM is projected onto a 2D cartesian coordinate system (here NAD83, the Canada Atlas Lambert projection). This is necessary to perform slope calculations. For more information on this, see: https://en.wikipedia.org/wiki/Map_projection

The DEM data returned in the process response here shows `rioxarray`-like access but using the URLs we can open the files however we like.

In [ ]:
terrain_resp = wps.terrain_analysis(
    shape=feature_url, 
    select_all_touching=True, 
    projected_crs=3978
)

In [ ]:
properties, dem = terrain_resp.get(asobj=True)

elevation = properties[0]["elevation"]
slope = properties[0]["slope"]
aspect = properties[0]["aspect"]

terrain = {
    "elevation": elevation, 
    "slope": slope, 
    "aspect": aspect
}
display(terrain)

In [ ]:
crs = ccrs.LambertConformal(
    central_latitude=49, 
    central_longitude=-95, 
    standard_parallels=(49, 77)
)

dem.name = "Elevation"
dem.attrs["units"] = "m"
ax = plt.subplot(projection=crs)
dem.where(dem != -32768).sel(band=1).plot.imshow(
    ax=ax, 
    transform=crs, 
    cmap="gnuplot"
)
plt.show()

In [ ]:
# We can also access the files directly via their URLs:
properties, dem = terrain_resp.get(asobj=False)
display(properties, dem)

# Let's read the data from band=1 as numpy array
display(rasterio.open(dem).read(1))

## Overview

A synthesis of all watershed properties can be created by merging the various dictionaries created. This allows users to easily access any of these values, and to provide them to Raven as needed.

In [ ]:
all_properties = {**shape_info, **land_use, **terrain}
display(all_properties)